https://adventofcode.com/2022/day/19

In [11]:
import re
import time
from copy import deepcopy
from concurrent import futures

In [2]:
with open("data/19.txt") as fh:
    data = fh.read()

In [3]:
testdata = """\
Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian
"""

In [4]:
def load_data(data):
    L = []
    for line in data.strip().splitlines():
        bp, ore_ore, clay_ore, obs_ore, obs_clay, geode_ore, geode_obs = (
            int(i) for i in re.findall(r"[+-]?\d+", line)
        )
        D = {
            "bp": bp,
            "robots": {
                "ore": {"ore": ore_ore},
                "clay": {"ore": clay_ore},
                "obs": {"ore": obs_ore, "clay": obs_clay},
                "geode": {"ore": geode_ore, "obs": geode_obs},
            },
        }
        L.append(D)
    return L


load_data(testdata)

[{'bp': 1,
  'robots': {'ore': {'ore': 4},
   'clay': {'ore': 2},
   'obs': {'ore': 3, 'clay': 14},
   'geode': {'ore': 2, 'obs': 7}}},
 {'bp': 2,
  'robots': {'ore': {'ore': 2},
   'clay': {'ore': 3},
   'obs': {'ore': 3, 'clay': 8},
   'geode': {'ore': 3, 'obs': 12}}}]

In [5]:
bps = load_data(testdata)
bps

[{'bp': 1,
  'robots': {'ore': {'ore': 4},
   'clay': {'ore': 2},
   'obs': {'ore': 3, 'clay': 14},
   'geode': {'ore': 2, 'obs': 7}}},
 {'bp': 2,
  'robots': {'ore': {'ore': 2},
   'clay': {'ore': 3},
   'obs': {'ore': 3, 'clay': 8},
   'geode': {'ore': 3, 'obs': 12}}}]

In [6]:
bps[0]['robots']

{'ore': {'ore': 4},
 'clay': {'ore': 2},
 'obs': {'ore': 3, 'clay': 14},
 'geode': {'ore': 2, 'obs': 7}}

In [7]:
def collect_geodes_dfs(bp, initial_state=None, max_et=24):
    if initial_state is None:
        initial_state = {"et": 0, "robots": {"ore": 1}, "minerals": {}}
    stack = [initial_state]
    best_count = 0
    best_fv = 0

    while stack:
        state = stack.pop()
        if max_future_value(state, max_et) < best_fv:
            continue

        geode_count = get_geode_count(state)
        if geode_count > best_count:
            best_count = geode_count
            fv = min_future_value(state, max_et)
            best_fv = max(fv, best_fv)

        et = state.get("et", 0)
        if et == max_et:
            continue
        et += 1

        robots = state.get("robots", {})
        minerals = state.get("minerals", {})

        # let it ride
        new_state = {"et": et, "robots": robots.copy(), "minerals": minerals.copy()}
        bots_work(new_state)
        stack.append(new_state)

        for bot in ["ore", "clay", "obs", "geode"]:
            new_state = {"et": et, "robots": robots.copy(), "minerals": minerals.copy()}
            requirements = bp["robots"][bot]
            diff = dict_subtract(new_state["minerals"], requirements)
            if all(v >= 0 for v in diff.values()):
                new_state["minerals"] = diff
                bots_work(new_state)
                new_state["robots"][bot] = new_state["robots"].get(bot, 0) + 1
                stack.append(new_state)

    return bp["bp"], best_count


def bots_work(state):
    for bot, botcount in state["robots"].items():
        state["minerals"][bot] = state["minerals"].get(bot, 0) + botcount


def dict_subtract(a, b):
    c = deepcopy(a)
    for k, v in b.items():
        c[k] = c.get(k, 0) - v
    return c


def get_geode_count(state):
    return state.get("minerals", {}).get("geode", 0)


def max_future_value(state, max_et=24):
    geodes = state["minerals"].get("geode", 0)
    geode_bots = state["robots"].get("geode", 0)
    for _ in range(state["et"], max_et):
        geodes += geode_bots
        geode_bots += 1
    return geodes


def min_future_value(state, max_et=24):
    geodes = state["minerals"].get("geode", 0)
    geode_bots = state["robots"].get("geode", 0)
    for _ in range(state["et"], max_et):
        geodes += geode_bots
    return geodes

In [8]:
%%time
collect_geodes_dfs(bps[0])

CPU times: user 3min 32s, sys: 11.1 ms, total: 3min 32s
Wall time: 3min 32s


(1, 9)

In [12]:
%%time
L = []
print(time.ctime())
with futures.ProcessPoolExecutor(4) as ex:
    todos = (ex.submit(collect_geodes_dfs, bp) for bp in bps)
    for done in futures.as_completed(todos):
        r = done.result()
        print(time.ctime(), r)
        L.append(r)
L

Tue Dec 27 11:30:23 2022
Tue Dec 27 11:34:10 2022 (1, 9)
Tue Dec 27 11:52:34 2022 (2, 12)
CPU times: user 31.8 ms, sys: 13.8 ms, total: 45.6 ms
Wall time: 22min 10s


[(1, 9), (2, 12)]

In [13]:
%%time
bps = load_data(data)
L = []
print(time.ctime())
with futures.ProcessPoolExecutor(4) as ex:
    todos = (ex.submit(collect_geodes_dfs, bp) for bp in bps)
    for done in futures.as_completed(todos):
        r = done.result()
        print(time.ctime(), r)
        L.append(r)
L

Tue Dec 27 11:55:57 2022
Tue Dec 27 12:20:33 2022 (4, 5)
Tue Dec 27 12:24:52 2022 (3, 1)
Tue Dec 27 12:25:13 2022 (5, 3)
Tue Dec 27 12:45:32 2022 (1, 0)
Tue Dec 27 12:50:18 2022 (6, 2)
Tue Dec 27 13:14:51 2022 (8, 1)
Tue Dec 27 13:15:01 2022 (7, 0)
Tue Dec 27 13:28:00 2022 (11, 1)
Tue Dec 27 13:28:57 2022 (9, 14)
Tue Dec 27 13:49:42 2022 (13, 5)
Tue Dec 27 13:52:25 2022 (14, 6)
Tue Dec 27 14:10:14 2022 (12, 0)
Tue Dec 27 14:55:18 2022 (16, 0)
Tue Dec 27 15:29:35 2022 (17, 1)


KeyboardInterrupt: 